<a href="https://colab.research.google.com/github/diegozamorasanchez/03MIAR-Algoritmos-de-Optimizacion/blob/main/Algoritmos_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Diego Zamora Sanchez <br>
Link: https://colab.research.google.com/drive/1zL5nXwOblhWQo8xSTQK-lE5VMZ-0yo3K?usp=sharing <br>
GitHub: https://github.com/diegozamorasanchez/03MIAR-Algoritmos-de-Optimizacion <br>

1. EL PROBLEMA DEL AGENTE VIAJERO

In [66]:
#CARGA DE LIBRERIAS
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
#CARGAMOS LOS DATOS DEL PROBLEMA
import urllib.request #Para llamadas http
import tsplib95       #Para instancias del problema del TSP
import math           #Funciones matematicas
import random         #Generadores de valores aleatorios

#Descargamos el fichero con la Matriz de Distancias
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprime

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [68]:
#Carga de datos descomprimidas y generación del objeto (problema del viajante)
problem = tsplib95.load(file)
#Nodos
Nodos = list(problem.get_nodes())
#Aristas
Aristas = list(problem.get_edges())

In [69]:
#Probamos algunas funciones del objeto problema

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones en la documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html               
#dir(problem)

15

In [70]:
#FUNCION PARA GENERAR SOLUCION ALEATORIA
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#FUNCION DISTANCIA ENTRE 2 NODOS
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#FUNCION DISTANCIA TOTAL DE UNA SOLUCION O TRAYECTORIA
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [71]:
solucion = crear_solucion(Nodos)
solucion

[0,
 38,
 20,
 8,
 9,
 26,
 2,
 13,
 39,
 1,
 16,
 37,
 11,
 35,
 21,
 40,
 15,
 4,
 25,
 27,
 28,
 29,
 36,
 30,
 24,
 12,
 5,
 17,
 7,
 19,
 18,
 6,
 14,
 3,
 10,
 32,
 31,
 23,
 41,
 34,
 33,
 22]

In [72]:
distancia_total(solucion, problem)

4462

2. BÚSQUEDA ALEATORIA

In [73]:
#FUNCION PRINCIPAL DE BUSQUEDA ALEATORIA
def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())

  mejor_solucion=[]
  mejor_distancia=float('inf')

  for i in range(N):
    solucion=crear_solucion(Nodos)
    distancia=distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion=solucion
      mejor_distancia=distancia

  print("Mejor solución:" , mejor_solucion)
  print("Distancia:" , mejor_distancia)
  return mejor_solucion

In [74]:
#Busqueda aleatoria con N=10000 iteraciones
solucion = busqueda_aleatoria(problem,10000)

Mejor solución: [0, 31, 34, 33, 26, 21, 22, 19, 41, 40, 24, 9, 27, 4, 6, 5, 15, 18, 32, 14, 2, 29, 39, 3, 8, 25, 28, 37, 35, 36, 38, 23, 11, 13, 16, 12, 20, 30, 10, 1, 7, 17]
Distancia: 3706


3. BÚSQUEDA LOCAL

In [75]:
#FUNCION DE BUSQUEDA LOCAL CON GENERADOR DE SOLUCIONES VECINAS
def genera_vecina(solucion):
  mejor_solucion=[]
  mejor_distancia=10e100
  #Recorremos todos los nodos
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      
      #a: se genera una nueva solución intercambiando los dos nodos i,j
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #b: se evalua la solución
      distancia_vecina=distancia_total(vecina, problem)    

      #c: Se guarda si es mejor
      if distancia_vecina <= mejor_distancia:
        mejor_distancia=distancia_vecina
        mejor_solucion=vecina

  return mejor_solucion

In [76]:
#solucion=[1,47,13,41,40,19,42,44,37,5,22,28,3,2,29,21,50,34,30,9,16,11,38,49,10,39,33,45,15,24,43,26,31,36,35,20,8,7,23,48,27,12,17,4,18,25,14,6,51,46,32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))

nueva_solucion=genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3706
Distancia Mejor Solucion Local: 3468


In [77]:
#FUNCION DE BUSQUEDA LOCAL SOBRE LA ANTERIOR SIN CRITERIO DE PARADA
def busqueda_local(problem):
  mejor_solucion=[]

  #Solucion inicial aleatoria
  solucion_referencia=crear_solucion(Nodos)
  mejor_distancia=distancia_total(solucion_referencia, problem)

  iteracion=0
  while(1):
    iteracion +=1

    #a: mejor vecina
    vecina=genera_vecina(solucion_referencia)

    #b:se evalúa
    distancia_vecina=distancia_total(vecina, problem)

    #c: se para si no mejoramos (mínimo local)
    if distancia_vecina < mejor_distancia:
      mejor_solucion=vecina
      mejor_distancia=distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia:" , mejor_distancia)
      return mejor_solucion

In [78]:
#Solución búsqueda local:
sol=busqueda_local(problem)

En la iteracion  2 , la mejor solución encontrada es: [0, 8, 21, 39, 40, 3, 22, 32, 11, 13, 20, 18, 16, 1, 15, 23, 31, 33, 35, 34, 27, 25, 24, 12, 37, 36, 30, 7, 19, 38, 2, 14, 26, 6, 28, 29, 5, 4, 10, 17, 9, 41]
Distancia: 4436


4. SIMULATED ANNEALING

In [79]:
#FUNCION GENERADORA DE SOLUCIONES VECINAS 100% ALEATORIA
def genera_vecina_aleatorio(solucion):

  #2 nodos aleatorios
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución intercambiando los 2 nodos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

#FUNCION PROBABILIDAD ACEPTACION
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T):
    return True
  else:
    return False

#FUNCION DESCENSO TEMPERATURA
def bajar_temperatura(T):
  return T*0.99

In [80]:
def recocido_simulado(problem, TEMPERATURA):

  solucion_referencia=crear_solucion(Nodos)
  distancia_referencia=distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera solución vecina
    vecina=genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula distancia
    distancia_vecina=distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre)
    if distancia_vecina < mejor_distancia:
        mejor_solucion=vecina
        mejor_distancia=distancia_vecina
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según función de probabilidad
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia=vecina
      distancia_referencia=distancia_vecina

    #Baja temperatura
    TEMPERATURA=bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

In [81]:
problem=tsplib95.load(file)
sol=recocido_simulado(problem,10000000)

La mejor solución encontrada es [0, 7, 31, 17, 37, 15, 14, 19, 5, 32, 34, 33, 30, 2, 8, 29, 9, 23, 12, 11, 25, 10, 41, 21, 39, 22, 28, 27, 6, 1, 4, 3, 18, 26, 13, 16, 36, 35, 20, 38, 24, 40]
con una distancia total de 2122


SALEN MAS CORTAS(OPTIMAS) LAS TRAYECTORIAS(SOLUCIONES) DE SIMULATED ANNEALING!